In [1]:
import numpy as np
import torch
from torchvision import transforms
from tifffile import imread
import sys
sys.path.append('../')
from noise_model.PixelCNN import PixelCNN
from HDN.models.lvae import LadderVAE
import os
from utils.dataloaders import create_dn_loader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Load noisy images

In [3]:
observation_location = '../data/conv/observation.tif'
observation = imread(observation_location)

Load trained noise model and disable gradients

In [4]:
noise_model_location = '../nm_checkpoint/conv/final_params.ckpt'
noise_model = PixelCNN.load_from_checkpoint(noise_model_location).eval().to(device)

for param in noise_model.parameters():
    param.requires_grad = False

Create data loaders and get the shape, mean and standard deviation of the noisy images

In [5]:
transform = None
dn_train_loader, dn_val_loader, img_shape, data_mean, data_std = create_dn_loader(observation, batch_size=8, split=0.8, transform=transform)

Set denoiser checkpoint directory

In [6]:
dn_checkpoint_path = '../dn_checkpoint/conv'
if not os.path.exists(dn_checkpoint_path):
    os.makedirs(dn_checkpoint_path)

Initialise trainer and VAE

In [7]:
num_latents = 6
z_dims = [32]*int(num_latents)

trainer = pl.Trainer(default_root_dir=dn_checkpoint_path,
                     accelerator='gpu',
                     devices=1,
                     max_steps=200000,
                     logger=TensorBoardLogger(dn_checkpoint_path),
                     callbacks=[ModelCheckpoint(save_weights_only=False, mode="min", monitor="val_elbo")])

vae = LadderVAE(z_dims=z_dims,
                noiseModel=noise_model,
                img_shape=img_shape,
                gaussian_noise_std=None,
                use_uncond_mode_at=[],
                dropout=0.2,
                free_bits=0.5,
                data_mean=data_mean,
                data_std=data_std).to(device)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/ben/miniconda3/envs/pl/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:261: UserWarning: Attribute 'noiseModel' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['noiseModel'])`.
  rank_zero_warn(


Train and save final parameters

In [ ]:
trainer.fit(vae, dn_train_loader, dn_val_loader)
trainer.save_checkpoint(os.path.join(dn_checkpoint_path, 'final_params.ckpt'))

Missing logger folder: ../dn_checkpoint/conv/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name             | Type                 | Params
----------------------------------------------------------
0 | first_bottom_up  | Sequential           | 75.8 K
1 | top_down_layers  | ModuleList           | 4.1 M 
2 | bottom_up_layers | ModuleList           | 2.7 M 
3 | final_top_down   | Sequential           | 412 K 
4 | likelihood       | NoiseModelLikelihood | 1.9 M 
----------------------------------------------------------
7.3 M     Trainable params
1.9 M     Non-trainable params
9.2 M     Total params
36.924    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/ben/miniconda3/envs/pl/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 36 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/ben/miniconda3/envs/pl/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 36 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/ben/miniconda3/envs/pl/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1933: PossibleUserWarning: The number of training batches (6) is smaller than the logging in

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]